In [1]:
import pandas as pd
import re
from io import StringIO
import numpy as np
import base64

In [2]:
df = pd.read_csv('labelled_enron.csv')
df

,column_a,message_id,date,from,to,subject,x_from,x_to,x_cc,x_bcc,...,cat_10_level_1,cat_10_level_2,cat_10_weight,cat_11_level_1,cat_11_level_2,cat_11_weight,cat_12_level_1,cat_12_level_2,cat_12_weight,labeled
0,379,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15T14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,381,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15T14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2139,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20T17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,2140,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20T17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2232,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09T12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,499650,<18361957.1075861368310.JavaMail.evans@thyme>,2001-11-05T13:42:45,frozenset({'louise.kitchen@enron.com'}),"frozenset({'greg.whalley@enron.com', 'john.lav...",Confidential Due Diligence Info - GS,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1698,501042,<29049822.1075842045435.JavaMail.evans@thyme>,2001-06-13T14:00:00,frozenset({'steven.kean@enron.com'}),"frozenset({'greg.whalley@enron.com', 'kenneth....",California's dim bulbs,Steven J Kean,"Kenneth Lay, Jeff Skilling, Andrew S Fastow, G...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1699,507376,<205897.1075861997314.JavaMail.evans@thyme>,2001-11-13T22:20:01,frozenset({'jeanie.slone@enron.com'}),frozenset({'mark.whitt@enron.com'}),RE: confidential ee info,"Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1700,509660,<23743848.1075863311776.JavaMail.evans@thyme>,2001-07-11T15:29:22,frozenset({'legalonline-compliance@enron.com'}),"frozenset({'williams@mailman.enron.com', 'bwil...",Confidential Information and Securities Trading,Office of the Chairman - Enron Wholesale Servi...,"WILLIAMS@mailman.enron.com, WILLIAM <bwillia5@...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [3]:
df=df[~df['content'].isna()]
df=df[df['labeled']==True]
col=['message_id','subject','content','cat_1_level_2']
df=df.loc[:,col]
df.head()

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,1.0
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,3.0
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",4.0
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",4.0
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",4.0


In [4]:
# Create a new 'label' column based on 'cat_1_level_2'
df['label'] = df['cat_1_level_2'].apply(lambda x: 1.0 + (x / 10.0))

# Map 'label' values to the desired labels
label_mapping = {
    1.1: 'Company Business, Strategy, etc.(elaborate in Section 3 [Topics])',
    1.2: 'Purely Personal',
    1.3: 'Personal but in professional context(e.g., it was good working with you)',
    1.4: 'Logistic Arrangements(meeting scheduling, technical support, etc)',
    1.5: 'Employment arrangements(job seeking, hiring, recommendations, etc) ',
    1.6: 'Document editing/checking(collaboration)',
    1.7: 'Empty message(due to missing attachment)',
    1.8: 'Empty message'
}

df['label'] = df['label'].map(label_mapping)
df.drop(columns=['cat_1_level_2'], inplace=True)
df

,message_id,subject,content,label
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,"Company Business, Strategy, etc.(elaborate in ..."
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,"Personal but in professional context(e.g., it ..."
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...","Logistic Arrangements(meeting scheduling, tech..."
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...","Logistic Arrangements(meeting scheduling, tech..."
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...","Logistic Arrangements(meeting scheduling, tech..."
...,...,...,...,...
1697,<18361957.1075861368310.JavaMail.evans@thyme>,Confidential Due Diligence Info - GS,There is a lot of confidential information her...,"Company Business, Strategy, etc.(elaborate in ..."
1698,<29049822.1075842045435.JavaMail.evans@thyme>,California's dim bulbs,You may have seen this when it first came out ...,"Company Business, Strategy, etc.(elaborate in ..."
1699,<205897.1075861997314.JavaMail.evans@thyme>,RE: confidential ee info,"Mark, Can you email me or fax me any written d...","Employment arrangements(job seeking, hiring, r..."
1700,<23743848.1075863311776.JavaMail.evans@thyme>,Confidential Information and Securities Trading,"To:WILLIAMS, WILLIAM Email:bwillia5@enron.com ...","Company Business, Strategy, etc.(elaborate in ..."


In [5]:
df.rename(columns={'content': 'body', 'label': 'category'}, inplace=True)
df.to_csv('simplified_labeled_enron.csv', index=False)
df.head()

,message_id,subject,body,category
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,"Company Business, Strategy, etc.(elaborate in ..."
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,"Personal but in professional context(e.g., it ..."
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...","Logistic Arrangements(meeting scheduling, tech..."
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...","Logistic Arrangements(meeting scheduling, tech..."
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...","Logistic Arrangements(meeting scheduling, tech..."


In [6]:
# Split into train and test dataset
original_df = pd.read_csv('simplified_labeled_enron.csv')

# Shuffle the rows randomly
shuffled_df = original_df.sample(frac=1, random_state=42)  # Use a fixed random_state for reproducibility

# Split the DataFrame into two parts
train_df = shuffled_df.iloc[:1400]
test_df = shuffled_df.iloc[1400:]

# Save the two split DataFrames to separate CSV files
train_df.to_csv('simplified_labeled_enron_train.csv', index=False)
test_df.to_csv('simplified_labeled_enron_test.csv', index=False)

In [7]:
# creating prompts for fine-tuning the model
train_df['prompts'] = "<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> [EMail-Subject]= " + train_df['subject'] + "; [EMail-Body]= " + train_df['body'] + " [/INST] [EMail-Category]= " + train_df['category']
train_df['prompts'][0]

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_74681/3130469722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['prompts'] = "<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> [EMail-Subject]= " +

"<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> [EMail-Subject]= Re: Confidential Employee Information/Lenhart; [EMail-Body]= I also need to know the base salaries of Jay Reitmeyer and Monique Sanchez. They are doing the same job as Matt. [/INST] [EMail-Category]= Company Business, Strategy, etc.(elaborate in Section 3 [Topics])"

In [8]:
# creating training dataset

prompts_df = pd.DataFrame(columns=['prompts'])
prompts_df['prompts'] = train_df['prompts'].astype(str)
prompts_df = prompts_df.drop(prompts_df[prompts_df['prompts'] == 'nan'].index)
prompts_df.reset_index()
prompts_df.to_csv('prompts_train.csv', index=False)
prompts_df

,prompts
618,<s>[INST] <<SYS>> I am sharing an email body a...
115,<s>[INST] <<SYS>> I am sharing an email body a...
135,<s>[INST] <<SYS>> I am sharing an email body a...
350,<s>[INST] <<SYS>> I am sharing an email body a...
1395,<s>[INST] <<SYS>> I am sharing an email body a...
...,...
509,<s>[INST] <<SYS>> I am sharing an email body a...
1135,<s>[INST] <<SYS>> I am sharing an email body a...
1250,<s>[INST] <<SYS>> I am sharing an email body a...
1416,<s>[INST] <<SYS>> I am sharing an email body a...


In [9]:
prompts_df[prompts_df['prompts']=='nan']

,prompts


In [10]:
# creating testing dataset
test_df['input_test_prompts'] = "<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> [EMail-Subject]= " + test_df['subject'] + "; [EMail-Body]= " + test_df['body'] + " [/INST] [EMail-Category]= "

prompts_df_test = pd.DataFrame(columns=['input_test_prompts', 'category'])
prompts_df_test['input_test_prompts'] = test_df['input_test_prompts'].astype(str)
prompts_df_test['category'] = test_df['category'].astype(str)
prompts_df_test = prompts_df_test.drop(prompts_df_test[prompts_df_test['input_test_prompts'] == 'nan'].index)
prompts_df_test.reset_index()
prompts_df_test.to_csv('prompts_test.csv', index=False)
prompts_df_test

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_74681/3341356277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['input_test_prompts'] = "<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> [EMail-Sub

,input_test_prompts,category
1611,<s>[INST] <<SYS>> I am sharing an email body a...,"Company Business, Strategy, etc.(elaborate in ..."
1143,<s>[INST] <<SYS>> I am sharing an email body a...,"Personal but in professional context(e.g., it ..."
53,<s>[INST] <<SYS>> I am sharing an email body a...,"Company Business, Strategy, etc.(elaborate in ..."
151,<s>[INST] <<SYS>> I am sharing an email body a...,"Logistic Arrangements(meeting scheduling, tech..."
403,<s>[INST] <<SYS>> I am sharing an email body a...,"Personal but in professional context(e.g., it ..."
...,...,...
1130,<s>[INST] <<SYS>> I am sharing an email body a...,"Company Business, Strategy, etc.(elaborate in ..."
1294,<s>[INST] <<SYS>> I am sharing an email body a...,"Company Business, Strategy, etc.(elaborate in ..."
860,<s>[INST] <<SYS>> I am sharing an email body a...,"Logistic Arrangements(meeting scheduling, tech..."
1459,<s>[INST] <<SYS>> I am sharing an email body a...,"Company Business, Strategy, etc.(elaborate in ..."


Checking a few prompts...

In [11]:
prompts_df_test['input_test_prompts'][1459]

"<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> [EMail-Subject]= Translation of articles; [EMail-Body]= ---------------------- Forwarded by Steven J Kean/HOU/EES on 10/18/99 08:46 AM --------------------------- Karen Denne@ENRON 10/18/99 07:08 AM To: Steven J Kean/HOU/EES@EES cc: Subject: Translation of articles ---------------------- Forwarded by Karen Denne/Corp/Enron on 10/18/99 07:07 AM --------------------------- Andrew Morrison@ECT 10/18/99 03:43 AM To

In [12]:
prompts_df_test['input_test_prompts'][1126]

'<s>[INST] <<SYS>> I am sharing an email body and its subject with you. Based on the text in the body and the subject, you need to classify the email in only one of the following eight categories: \'Company Business, Strategy, etc.\'; \'Purely Personal\'; \'Personal but in professional context (e.g., it was good working with you)\'; \'Logistic Arrangements (meeting scheduling, technical support, etc)\'; \'Employment arrangements (job seeking, hiring, recommendations, etc)\'; \'Document editing/checking (collaboration)\'; \'Empty message (due to missing attachment)\'; \'Empty message\'. <</SYS>> [EMail-Subject]= Re: CI at EBS; [EMail-Body]= Thanks for the message. Rita Hartfield@ENRON COMMUNICATIONS 07/17/2000 02:52 PM To: Steven J Kean/HOU/EES@EES cc: Anthony Mends/Enron Communications@Enron Communications Subject: CI at EBS Steve, I have heard "through the grapevine" that Amy Oberg has relayed to you that EBS will not be doing competitive intelligence and that you were dismayed to hea